In [ ]:
### Modelling ###

In [13]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle

import types
from botocore.client import Config
import ibm_boto3

In [2]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3b08f636aa4c4bc58b03e36105622462 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='EI06iPS11MRoHQiLAPwMFIukCpW_Yx5PqbVhI0FSYqjZ',
    ibm_auth_endpoint="https://iam.eu-de.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_3b08f636aa4c4bc58b03e36105622462.get_object(Bucket='courserabadges-donotdelete-pr-72y9f6bzbckngj',Key='online_shoppers_intention_eng.csv')['Body']
body_tf = client_3b08f636aa4c4bc58b03e36105622462.get_object(Bucket='courserabadges-donotdelete-pr-72y9f6bzbckngj',Key='online_shoppers_intention_eng_tf.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df_tf = pd.read_csv(body_tf)

In [3]:
# deep neural network
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# create model
def create_NN():
    model = Sequential()
    model.add(Dense(50, input_dim=74, activation='relu'))
    model.add(Dropout(rate=0.4))
    model.add(Dense(80, input_dim=50, activation='relu'))
    model.add(Dense(50, input_dim=50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

models = {
    'LogisticRegression': LogisticRegression(solver='lbfgs', max_iter = 500),
    'GradientBoosting': GradientBoostingClassifier(), 
    'RandomForest': RandomForestClassifier(),
    'NN': KerasClassifier(build_fn=create_NN, epochs=20, batch_size=10, verbose=0)
}
models = {
    'NN': KerasClassifier(build_fn=create_NN, epochs=10, batch_size=100, verbose=1)
}

models = {
    'LogisticRegression': LogisticRegression(solver='lbfgs', max_iter = 500)
}

feature_sets = {'data': df, 'df_tf': df_tf}

Using TensorFlow backend.


In [4]:
results = list()
for data_name, data in feature_sets.items():
    X = data.iloc[:,:-1]
    y = data.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = model.score(X_test, y_test)
        f1 = f1_score(y_true=y_test, y_pred=y_pred)
        results.append([data_name, model_name, model, f1, accuracy])
        

In [5]:
for result in results:
    print(result)

['data', 'LogisticRegression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False), 0.5097588978185994, 0.8841248303934871]
['df_tf', 'LogisticRegression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False), 0.6039215686274509, 0.8903663500678426]


In [6]:
clf = results[1][2]

In [7]:
#result = zip(df.columns, clf.feature_importances_)
#result = sorted(result, key=lambda x: x[1], reverse=True)
#result[:10]

AttributeError: 'LogisticRegression' object has no attribute 'feature_importances_'

In [8]:
# concentrate feature eng on page values, multiply with others for example...

In [23]:
deployment_model = results[0][2]
final_model = pickle.dumps(deployment_model)
info = client_3b08f636aa4c4bc58b03e36105622462.put_object(Bucket='courserabadges-donotdelete-pr-72y9f6bzbckngj',Key='final_model.pkl', Body=final_model)

{'ResponseMetadata': {'RequestId': 'e802703c-f956-4a96-9a58-d937975912f1',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 11 May 2020 09:41:18 GMT',
   'x-clv-request-id': 'e802703c-f956-4a96-9a58-d937975912f1',
   'server': 'Cleversafe/3.14.11.35',
   'x-clv-s3-version': '2.5',
   'x-amz-request-id': 'e802703c-f956-4a96-9a58-d937975912f1',
   'etag': '"140ca2bdfbd7ea9e1de00f7444cf58b0"',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"140ca2bdfbd7ea9e1de00f7444cf58b0"'}